Что делать?  

1.Датасет ml-latest  
2.Вспомнить подходы, которые мы разбирали  
3.Выбрать понравившийся подход к гибридным системам  
4.Написать свою  

Материалы здесь: https://github.com/ALKONDR/netology-recsys/blob/master/lecture-5/lecture-5-part-2.ipynb

In [1]:

from surprise import Dataset
from surprise import accuracy
from surprise import Reader
from surprise.model_selection import train_test_split

import matplotlib.pyplot as plt

from tqdm import tqdm_notebook

from sklearn.feature_extraction.text import TfidfTransformer, CountVectorizer
from sklearn.neighbors import NearestNeighbors

import pandas as pd
import numpy as np

In [2]:
links = pd.read_csv('links.csv')
movies = pd.read_csv('movies.csv')
ratings = pd.read_csv('ratings.csv')
tags = pd.read_csv('tags.csv')

In [3]:
movies_with_ratings = movies.join(ratings.set_index('movieId'), on='movieId').reset_index(drop=True)
movies_with_ratings.dropna(inplace=True)

In [5]:
# movies_with_ratings.head()

In [46]:
dataset = pd.DataFrame({
    'uid': movies_with_ratings.userId,
    'iid': movies_with_ratings.title,
    'rating': movies_with_ratings.rating
})

In [47]:
reader = Reader(rating_scale=(0.5, 5.0))
data = Dataset.load_from_df(dataset, reader)

In [48]:
trainset, testset = train_test_split(data, test_size=.15, random_state=42)

# learning

In [71]:
from surprise import SVD

In [72]:
%%time
algo_1 = SVD(n_factors=20, n_epochs=20)
algo_1.fit(trainset)

Wall time: 2.31 s


In [73]:
test_pred_1 = algo_1.test(testset)

In [74]:
accuracy.rmse(test_pred_1, verbose=True)

RMSE: 0.8708


0.8708154840990902

In [75]:
from surprise import KNNWithMeans

In [76]:
# %%time
algo_2 = KNNWithMeans(k=20, sim_options={'name': 'pearson_baseline', 'user_based': False})
algo_2.fit(trainset)

Estimating biases using als...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.
Wall time: 17.3 s


In [77]:
test_pred_2 = algo_2.test(testset)

In [78]:
accuracy.rmse(test_pred_2, verbose=True)

RMSE: 0.8740


0.8739785042204141

In [79]:
from surprise import SVDpp

In [80]:
# %%time
algo_3 = SVDpp()
algo_3.fit(trainset)

Wall time: 11min 57s


In [81]:
test_pred_3 = algo_3.test(testset)

In [82]:
accuracy.rmse(test_pred_3, verbose=True)

RMSE: 0.8583


0.8583040210997777

In [83]:
from surprise import SlopeOne

In [84]:
# %%time
algo_4 = SlopeOne()
algo_4.fit(trainset)

Wall time: 5.76 s


In [85]:
test_pred_4 = algo_4.test(testset)

In [86]:
accuracy.rmse(test_pred_4, verbose=True)

RMSE: 0.8959


0.8959120981252762

# stacking

In [122]:
predicts_df  = dataset.copy().drop(['uid','iid'], axis=1)

for tqdm_notebook(index, row in dataset.iterrows()):
    predicts_df.loc[index,'algo_1'] = algo_1.predict(uid=row['uid'], iid=row['iid']).est
    predicts_df.loc[index,'algo_2'] = algo_2.predict(uid=row['uid'], iid=row['iid']).est
    predicts_df.loc[index,'algo_3'] = algo_3.predict(uid=row['uid'], iid=row['iid']).est
    predicts_df.loc[index,'algo_4'] = algo_4.predict(uid=row['uid'], iid=row['iid']).est
#     break

In [123]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression

In [126]:
# predicts_df

In [128]:
X = predicts_df.drop('rating',axis=1)
y = predicts_df['rating']

In [129]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

In [130]:
algo_5 = LinearRegression()

In [131]:
algo_5.fit(X_train, y_train)

LinearRegression()

In [133]:
final_pred = algo_5.predict(X_test)

In [134]:
from sklearn.metrics import mean_squared_error

In [135]:
rmse_final = np.sqrt(mean_squared_error(y_test, final_pred))

In [143]:
def stacking_recommendation(user_id, item_id):
    predictions = pd.DataFrame()
    
    predictions.loc[0,'algo_1'] = algo_1.predict(uid=row['uid'], iid=row['iid']).est
    predictions.loc[0,'algo_2'] = algo_2.predict(uid=row['uid'], iid=row['iid']).est
    predictions.loc[0,'algo_3'] = algo_3.predict(uid=row['uid'], iid=row['iid']).est
    predictions.loc[0,'algo_4'] = algo_4.predict(uid=row['uid'], iid=row['iid']).est
    
    fnal_est = algo_5.predict(predictions)
    
    return fnal_est

In [148]:
stacking_recommendation(184,'Toy Story (1995)')

array([3.95768502])

In [149]:
# предсказан рейтинг конкретного фильма для конкретного пользователя при помощи стакинга моделей

# метарекомендации

In [207]:
pred_df = predicts_df.copy()

In [214]:
pred_df = pred_df.join(dataset.drop('rating', axis=1))

In [215]:
pred_df.head()

,rating,algo_1,algo_2,algo_3,algo_4,uid,iid
0,4.0,4.631020,4.437411,4.743843,4.688279,1.0,Toy Story (1995)
1,4.0,3.810775,4.031528,3.877734,3.859359,5.0,Toy Story (1995)
2,4.5,3.441314,4.135260,3.368667,3.686063,7.0,Toy Story (1995)
3,2.5,3.684760,2.935026,3.397402,3.670467,15.0,Toy Story (1995)
4,4.5,4.082983,4.296729,4.247865,4.143816,17.0,Toy Story (1995)


In [236]:
best_algo_for_user_dict = {}

for user_id, group in tqdm_notebook(pred_df.groupby('uid')):
    rmse_list = []
    
    rmse_list.append(np.sqrt(mean_squared_error(group['rating'], group['algo_1'])))
    rmse_list.append(np.sqrt(mean_squared_error(group['rating'], group['algo_2'])))
    rmse_list.append(np.sqrt(mean_squared_error(group['rating'], group['algo_3'])))
    rmse_list.append(np.sqrt(mean_squared_error(group['rating'], group['algo_4'])))
    
    best_algo_for_user_index = np.array(rmse_list).argmin()
    
    best_algo_for_user_dict.setdefault(user_id,(f'algo_{best_algo_for_user_index+1}'))
#     break

<ipython-input-236-f67261a6bfba>:3: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for user_id, group in tqdm_notebook(pred_df.groupby('uid')):


In [273]:
models_dict = {
    'algo_1' : algo_1, 
    'algo_2' : algo_2, 
    'algo_3' : algo_3, 
    'algo_4' : algo_4, 
}

In [252]:
best_algo_for_user_dict.get(184)

'algo_2'

In [275]:
def best_algo_recommendation(user_id, item_id):
    best_algo = best_algo_for_user_dict.get(user_id)

    pred_rait = models_dict.get(best_algo).predict(user_id, item_id)
    
    return pred_rait

In [278]:
best_algo_recommendation(184,'Toy Story (1995)')

Prediction(uid=184, iid='Toy Story (1995)', r_ui=None, est=4.5419459385279914, details={'actual_k': 20, 'was_impossible': False})

In [279]:
# для каждого пользователя отобран "лучший"/наиболее точный алгоритм для предсказания
# по этому алгоритму делается итоговый прогноз для конкретного пользователя и фильма